<a href="https://colab.research.google.com/github/kjmobile/lb/blob/main/Ch10_deeplearning_lab_Part1_A_GPU_Colab_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning


In [ ]:
!pip install --upgrade pip
!pip install torchinfo torchvision torchmetrics
!pip install pytorch_lightning
!pip install islp

In [ ]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

### Torch-Specific Imports


In [ ]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset


In [ ]:
from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary


In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger


In [ ]:
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


In [ ]:
from torchvision.io import read_image
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50,
                                ResNet50_Weights)
from torchvision.transforms import (Resize,
                                    Normalize,
                                    CenterCrop,
                                    ToTensor)

In [ ]:
from ISLP.torch import (SimpleDataModule,SimpleModule, ErrorTracker, rec_num_workers)

In [ ]:
from ISLP.torch.imdb import (load_lookup,load_tensor,load_sparse,load_sequential)

In [ ]:
from glob import glob
import json


## Single Layer Network on Hitters Data


In [ ]:
Hitters = load_data('Hitters').dropna()
n = Hitters.shape[0]


In [ ]:
model = MS(Hitters.columns.drop('Salary'), intercept=False)
X = model.fit_transform(Hitters).to_numpy() ###ch
Y = Hitters['Salary'].to_numpy()


In [ ]:
(X_train,
 X_test,
 Y_train,
 Y_test) = train_test_split(X,
                            Y,
                            test_size=1/3,
                            random_state=1)

### Linear Models


In [ ]:
hit_lm = LinearRegression().fit(X_train, Y_train)
Yhat_test = hit_lm.predict(X_test)
np.abs(Yhat_test - Y_test).mean() ##check how much error in dolloars

In [ ]:
# same as below
# from sklearn.metrics import mean_absolute_error as mae
# mae(Yhat_test, Y_test)

In [ ]:
scaler = StandardScaler(with_mean=True, with_std=True)
lasso = Lasso(warm_start=True, max_iter=30000) #Q1
standard_lasso = Pipeline(steps=[('scaler', scaler),
                                 ('lasso', lasso)])

In [ ]:
X_s = scaler.fit_transform(X_train)
n = X_s.shape[0]
lam_max = np.fabs(X_s.T.dot(Y_train - Y_train.mean())).max() / n #Q2
param_grid = {'lasso__alpha': np.exp(np.linspace(0, np.log(0.01), 100))
             * lam_max}

In [ ]:
cv = KFold(10,
           shuffle=True,
           random_state=1)
grid = GridSearchCV(standard_lasso,
                    param_grid,
                    cv=cv,
                    scoring='neg_mean_absolute_error')
grid.fit(X_train, Y_train);

In [ ]:
trained_lasso = grid.best_estimator_
Yhat_test = trained_lasso.predict(X_test)
np.fabs(Yhat_test - Y_test).mean() #Q3 now how much mae  in dollar after applying cv on lasso?

### Specifying a Network: Classes and Inheritance - See illustration in c10-nn


In [ ]:
class HittersModel(nn.Module): #Q4, Q5

    def __init__(self, input_size):
        super(HittersModel, self).__init__() #Q6
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(input_size, 50),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(50, 1))

    def forward(self, x):
        x = self.flatten(x)
        return torch.flatten(self.sequential(x))


In [ ]:
hit_model = HittersModel(X.shape[1])


In [ ]:
summary(hit_model,                     #Q8, #Q9
        input_size=X_train.shape,
        col_names=['input_size',
                   'output_size',
                   'num_params'])    ### num params, why 1000. sequential(), hidden layer, output layer, Relu and Dropout are "operations" Q10 50 bias-terms


In [ ]:
X_train_t = torch.tensor(X_train.astype(np.float32))
Y_train_t = torch.tensor(Y_train.astype(np.float32))
hit_train = TensorDataset(X_train_t, Y_train_t)

In [ ]:
X_test_t = torch.tensor(X_test.astype(np.float32))
Y_test_t = torch.tensor(Y_test.astype(np.float32))
hit_test = TensorDataset(X_test_t, Y_test_t) #Q12


In [ ]:
max_num_workers = rec_num_workers()

In [ ]:
from ISLP.torch import (SimpleDataModule,SimpleModule, ErrorTracker, rec_num_workers)

hit_dm = SimpleDataModule(hit_train,    #Q11 #Q13
                          hit_test,
                          batch_size=32, #Q14
                          num_workers=min(4, max_num_workers),
                          validation=hit_test)

In [ ]:
hit_module = SimpleModule.regression(hit_model,
                           metrics={'mae':MeanAbsoluteError()}) # Q17


In [ ]:
hit_logger = CSVLogger('logs', name='hitters')

In [ ]:
from pytorch_lightning import Trainer

hit_trainer = Trainer(deterministic=False,    #Q15
                      max_epochs=50,
                      log_every_n_steps=5,
                      logger=hit_logger,
                      callbacks=[ErrorTracker()])
hit_trainer.fit(hit_module, datamodule=hit_dm) #  Q16

In [ ]:
hit_trainer.test(hit_module, datamodule=hit_dm) ### MAE now


In [ ]:
hit_logger.experiment.log_metrics

In [ ]:
hit_results = pd.read_csv(hit_logger.experiment.metrics_file_path)

In [ ]:
hit_results

In [ ]:
def summary_plot(results,
                 ax,
                 col='loss',
                 valid_legend='Validation',
                 training_legend='Training',
                 ylabel='Loss',
                 fontsize=20):
    for (column,
         color,
         label) in zip([f'train_{col}_epoch',
                        f'valid_{col}'],
                       ['black',
                        'red'],
                       [training_legend,
                        valid_legend]):
        results.plot(x='epoch',
                     y=column,
                     label=label,
                     marker='*',
                     color=color,
                     ax=ax)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(ylabel)
    return ax

In [ ]:
fig, ax = subplots(1, 1, figsize=(6, 6)) #Q18
ax = summary_plot(hit_results,
                  ax,
                  col='mae',
                  ylabel='MAE',
                  valid_legend='Validation (=Test)')
ax.set_ylim([0, 400])
ax.set_xticks(np.linspace(0, 50, 11).astype(int));

In [ ]:
hit_model.eval()
preds = hit_module(X_test_t)
torch.abs(Y_test_t - preds).mean() #Q19

### Cleanup


In [ ]:
del(Hitters,  #Q20
    hit_model, hit_dm,
    hit_logger,
    hit_test, hit_train,
    X, Y,
    X_test, X_train,
    Y_test, Y_train,
    X_test_t, Y_test_t,
    hit_trainer, hit_module)


## Multilayer Network on the MNIST Digit Data


In [ ]:
from torchvision.datasets import MNIST, CIFAR100

(mnist_train,                                #Q22, Q23. ### num records
 mnist_test) = [MNIST(root='data',
                      train=train,
                      download=True,
                      transform=ToTensor())
                for train in [True, False]]
mnist_train


In [ ]:
mnist_dm = SimpleDataModule(mnist_train,
                            mnist_test,
                            validation=0.2,
                            num_workers=max_num_workers,
                            batch_size=256)


In [ ]:
for idx, (X_ ,Y_) in enumerate(mnist_dm.train_dataloader()):
    print('X: ', X_.shape)
    print('Y: ', Y_.shape)
    if idx >= 1:
        break


# sample handwriting image per category

In [ ]:
import matplotlib.pyplot as plt

found = {}
for img, label in zip(mnist_train.data, mnist_train.targets):
    lbl = label.item()
    if lbl not in found:
        found[lbl] = img
    if len(found) == 10:
        break

fig, axes = plt.subplots(1, 10, figsize=(10, 1))
for i in range(10):
    axes[i].imshow(found[i], cmap='gray')
    axes[i].set_title(f"Label: {i}")
    axes[i].axis('off')
plt.tight_layout()


In [ ]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 256),  ### 256, 128 determined by... domain experience; rule of thusm: begin experiements with 256, 128, 64...
            nn.ReLU(),
            nn.Dropout(0.4))
        self.layer2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3))
        self._forward = nn.Sequential(
            self.layer1,
            self.layer2,
            nn.Linear(128, 10))
    def forward(self, x):
        return self._forward(x)

In [ ]:
mnist_model = MNISTModel()


In [ ]:
summary(mnist_model,
        input_data=X_,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

In [ ]:
mnist_module = SimpleModule.classification(mnist_model,num_classes=10)
mnist_logger = CSVLogger('logs', name='MNIST')


In [ ]:
mnist_trainer = Trainer(deterministic=False,
                        max_epochs=30,
                        logger=mnist_logger,
                        enable_progress_bar=True,
                        callbacks=[ErrorTracker()])
mnist_trainer.fit(mnist_module,
                  datamodule=mnist_dm)


In [ ]:
mnist_results = pd.read_csv(mnist_logger.experiment.metrics_file_path)
fig, ax = subplots(1, 1, figsize=(6, 6))
summary_plot(mnist_results,
             ax,
             col='accuracy',
             ylabel='Accuracy')
ax.set_ylim([0.5, 1])
ax.set_ylabel('Accuracy')
ax.set_xticks(np.linspace(0, 30, 7).astype(int));


In [ ]:
mnist_trainer.test(mnist_module,
                   datamodule=mnist_dm) #Q26.

###  Individual item prediction check :                            

In [ ]:
mnist_module.eval() # Set the module in evaluation mode.
test_loader = mnist_dm.test_dataloader()
X_batch, y_batch = next(iter(test_loader))

with torch.no_grad(): #Context-manager that disables  back propagating gradient calculation
    logits = mnist_module(X_batch)
    preds = torch.argmax(logits, dim=1)

print(f"Predicted: {preds[0].item()}, Actual: {y_batch[0].item()}")
if preds[0].item() == y_batch[0].item():
    print("Correct")
else:
    print("Wrong")

### show  pred vs. actual for 10 first samples from test set

In [ ]:
import matplotlib.pyplot as plt
import torch

with torch.no_grad():
    logits = mnist_module(X_batch)
    preds = torch.argmax(logits, dim=1)

num_samples = 10
plt.figure(figsize=(10, 2))
for i in range(num_samples):
    plt.subplot(1, num_samples, i + 1)
    plt.imshow(X_batch[i].squeeze(), cmap='gray')

    # Check correctness
    pred = preds[i].item()
    actual = y_batch[i].item()
    color = 'red' if pred != actual else 'black'

    plt.title(f"Pred: {pred}\nActual: {actual}", color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()


## Multiclass Logistic Regression (Not a Neural Net; no hidden layer, no activation function)

In [ ]:
class MNIST_MLR(nn.Module):
    def __init__(self):
        super(MNIST_MLR, self).__init__()
        self.linear = nn.Sequential(nn.Flatten(),
                                    nn.Linear(784, 10))
    def forward(self, x):
        return self.linear(x)

mlr_model = MNIST_MLR()
mlr_module = SimpleModule.classification(mlr_model,
                                         num_classes=10)
mlr_logger = CSVLogger('logs', name='MNIST_MLR')

In [ ]:
mlr_trainer = Trainer(deterministic=False,
                      max_epochs=30,
                      enable_progress_bar=True,
                      callbacks=[ErrorTracker()])
mlr_trainer.fit(mlr_module, datamodule=mnist_dm)

In [ ]:
mlr_trainer.test(mlr_module,
                 datamodule=mnist_dm)

#### 10 first samples from test set

In [ ]:
test_loader = mnist_dm.test_dataloader()
X_batch, y_batch = next(iter(test_loader))

mlr_module.eval()  # Set to evaluation mode
with torch.no_grad():
    logits = mlr_module(X_batch)               # Use the trained mlr_module
    preds = torch.argmax(logits, dim=1)        # Get predicted class index

num_samples = min(10, X_batch.shape[0])
plt.figure(figsize=(10, 2))
for i in range(num_samples):
    plt.subplot(1, num_samples, i + 1)
    plt.imshow(X_batch[i].squeeze(), cmap='gray')

    pred = preds[i].item()
    actual = y_batch[i].item()
    color = 'red' if pred != actual else 'black'

    plt.title(f"Pred: {pred}\nActual: {actual}", color=color, fontsize=8)
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
del(mnist_test,
    mnist_train,
    mnist_model,
    mnist_dm,
    mnist_trainer,
    mnist_module,
    mnist_results,
    mlr_model,
    mlr_module,
    mlr_trainer)

## Convolutional Neural Networks


In [ ]:
(cifar_train,
 cifar_test) = [CIFAR100(root="data",
                         train=train,
                         download=True)
             for train in [True, False]] #Q27

In [ ]:
cifar_train

In [ ]:
cifar_train.data.shape ### how many chanels

In [ ]:
transform = ToTensor() ### Convert image (H, W, C, uint8) to tensor (C, H, W, float32) scaled to [0,1]
cifar_train_X = torch.stack([transform(x) for x in
                            cifar_train.data])
cifar_test_X = torch.stack([transform(x) for x in
                            cifar_test.data])
cifar_train = TensorDataset(cifar_train_X,
                            torch.tensor(cifar_train.targets))
cifar_test = TensorDataset(cifar_test_X,
                            torch.tensor(cifar_test.targets))

In [ ]:
cifar_train_X.shape ### channels color

In [ ]:
cifar_dm = SimpleDataModule(cifar_train,
                            cifar_test,
                            validation=0.2,
                            num_workers=max_num_workers,
                            batch_size=128)


In [ ]:
for idx, (X_ ,Y_) in enumerate(cifar_dm.train_dataloader()): #train_dataloader() fetches the training data in batches.
    print('X: ', X_.shape)
    print('Y: ', Y_.shape)
    if idx >= 1: ### show only two batches
        break


In [ ]:
fig, axes = subplots(5, 5, figsize=(5,5))
rng = np.random.default_rng(4)
indices = rng.choice(np.arange(len(cifar_train)), 25,
                     replace=False).reshape((5,5))
for i in range(5):
    for j in range(5):
        idx = indices[i,j]
        axes[i,j].imshow(np.transpose(cifar_train[idx][0],
                                      [1,2,0]),
                                      interpolation=None)
        axes[i,j].set_xticks([])
        axes[i,j].set_yticks([])


In [ ]:
class BuildingBlock(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels):

        super(BuildingBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels,
                              kernel_size=(3,3),
                              padding='same')
        self.activation = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(2,2))

    def forward(self, x):
        return self.pool(self.activation(self.conv(x)))


In [ ]:
class CIFARModel(nn.Module): #Q30

    def __init__(self):
        super(CIFARModel, self).__init__()
        sizes = [(3,32),
                 (32,64),
                 (64,128),
                 (128,256)]
        self.conv = nn.Sequential(*[BuildingBlock(in_, out_)
                                    for in_, out_ in sizes])

        self.output = nn.Sequential(nn.Dropout(0.5),
                                    nn.Linear(2*2*256, 512),
                                    nn.ReLU(),
                                    nn.Linear(512, 100))
    def forward(self, x):
        val = self.conv(x)
        val = torch.flatten(val, start_dim=1)
        return self.output(val)


In [ ]:
cifar_model = CIFARModel()
summary(cifar_model,
        input_data=X_,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

In [ ]:
cifar_optimizer = RMSprop(cifar_model.parameters(), lr=0.001)
# In PyTorch, parameters() collects all the hidden weights and biases inside the model and passes them to the optimizer.
# This allows the optimizer to know exactly which parameters to update during training.

cifar_module = SimpleModule.classification(cifar_model,
                                    num_classes=100,
                                    optimizer=cifar_optimizer)
cifar_logger = CSVLogger('logs', name='CIFAR100')


In [ ]:
cifar_trainer = Trainer(deterministic=False,    # takes more than 10 minutes don't run again.
                        max_epochs=30,
                        logger=cifar_logger,
                        enable_progress_bar=True,
                        callbacks=[ErrorTracker()])
cifar_trainer.fit(cifar_module,
                  datamodule=cifar_dm)


In [ ]:
log_path = cifar_logger.experiment.metrics_file_path
cifar_results = pd.read_csv(log_path)
fig, ax = subplots(1, 1, figsize=(6, 6))
summary_plot(cifar_results,
             ax,
             col='accuracy',
             ylabel='Accuracy')
ax.set_xticks(np.linspace(0, 10, 6).astype(int))
ax.set_ylabel('Accuracy')
ax.set_ylim([0, 1]);

In [ ]:
cifar_trainer.test(cifar_module,
                   datamodule=cifar_dm)


### Hardware Acceleration :using apple M1/M2 GPU only if available


In [ ]:
# try:
#     for name, metric in cifar_module.metrics.items():
#         cifar_module.metrics[name] = metric.to('mps')
#     cifar_trainer_mps = Trainer(accelerator='mps',
#                                 deterministic=True,
#                                 enable_progress_bar=False,
#                                 max_epochs=30)
#     cifar_trainer_mps.fit(cifar_module,
#                           datamodule=cifar_dm)
#     cifar_trainer_mps.test(cifar_module,
#                           datamodule=cifar_dm)
# except:
#     pass

## Using Pretrained CNN Models


In [ ]:
import os, urllib.request, requests #added to create book_images folder
from glob import glob

os.makedirs("book_images", exist_ok=True)

r = requests.get("https://api.github.com/repos/kjmobile/data/contents/img/book_images")
urls = [f["download_url"] for f in r.json() if f["name"].lower().endswith((".png", ".jpg", ".jpeg"))]

for url in urls:
    urllib.request.urlretrieve(url, f"book_images/{url.split('/')[-1]}")




In [ ]:
resize = Resize((232,232), antialias=True)
crop = CenterCrop(224)
normalize = Normalize([0.485,0.456,0.406],
                      [0.229,0.224,0.225])
imgfiles = sorted(glob("book_images/*"))
imgs = torch.stack([torch.div(crop(resize(read_image(f))), 255)
                    for f in imgfiles])
imgs = normalize(imgs)
imgs.size()

In [ ]:
resnet_model = resnet50(weights=ResNet50_Weights.DEFAULT)
summary(resnet_model,
        input_data=imgs,
        col_names=['input_size',
                   'output_size',
                   'num_params'])


In [ ]:
resnet_model.eval()

In [ ]:
img_preds = resnet_model(imgs)


In [ ]:
img_probs = np.exp(np.asarray(img_preds.detach()))
img_probs /= img_probs.sum(1)[:,None]


In [ ]:
import urllib.request, json
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"   # imagenet download
urllib.request.urlretrieve(url, "imagenet_class_index.json")

with open("imagenet_class_index.json") as f:
    labs = json.load(f)


In [ ]:
labs = json.load(open('imagenet_class_index.json'))
class_labels = pd.DataFrame([(int(k), v[1]) for k, v in
                           labs.items()],
                           columns=['idx', 'label'])
class_labels = class_labels.set_index('idx')
class_labels = class_labels.sort_index()


In [ ]:
for i, imgfile in enumerate(imgfiles):
    img_df = class_labels.copy()
    img_df['prob'] = img_probs[i]
    img_df = img_df.sort_values(by='prob', ascending=False)[:3]
    print(f'Image: {imgfile}')
    print(img_df.reset_index().drop(columns=['idx']))


In [ ]:
del(cifar_test,
    cifar_train,
    cifar_dm,
    cifar_module,
    cifar_logger,
    cifar_optimizer,
    cifar_trainer)